# Primero, instala las librerías necesarias

In [1]:
!pip install numpy pandas scikit-learn transformers datasets sacrebleu rouge-score

In [2]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_metric
from datasets import load_dataset
import torch
import requests
import nltk
from IPython.display import display

# Descargar recursos de nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Cargar datos

In [3]:
# Posibles valores:     temp0.1    temp0.3   temp0.5   temp1
temperature = "temp0.1"

url = 'https://raw.githubusercontent.com/Beta-sebas/amadeus_trl_assistant_dataset/main/evaluation_fase_dataset_' + temperature + '.json'

# Obtener datos desde GitHub
response = requests.get(url)
data_json = response.json()

# Convertir el JSON en un DataFrame de Pandas
data = pd.DataFrame(data_json)

print(data.head())

                                              prompt  \
0  ¿Qué tipo de actividades se realizan en el TRL 1?   
1          ¿Qué actividades se realizan en el TRL 7?   
2      ¿Cómo se evidencia la finalización del TRL 8?   
3   ¿Qué tipo de resultados se obtienen en el TRL 6?   
4                 ¿Qué representa el nivel 5 de TRL?   

                                          referencia  \
0  En el TRL 1 se realizan actividades de investi...   
1  En el TRL 7 se realizan pruebas en condiciones...   
2  La finalización del TRL 8 se demuestra cuando ...   
3  Los principales resultados del TRL 6 son el pr...   
4  El TRL 5 representa la prueba del prototipo de...   

                                    generado_llama_2  \
0   En el TRL 1 se realizan actividades de invest...   
1   En el TRL 7 se realizan pruebas en condicione...   
2   La finalización del TRL 8 se demuestra cuando...   
3   Los resultados del TRL 6 son prototipos de pr...   
4   El TRL 5 representa la validación de los c

# Herramientas

# Funciones para calcular métricas

Se calculan:
-Perplejidad
-exactitud
-precisión
-recuperación,
-la medida F1
-BLEU
-ROUGE.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def calculate_perplexity(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

def evaluate_model_individual(data, model_name):
    references = data['referencia'].tolist()
    predictions = data[f'generado_{model_name}'].tolist()
    inference_time = data[f'tiempo_ejecucion_{model_name}'].tolist()

    results = []

    for ref, pred, inf in zip(references, predictions, inference_time):
        # Perplejidad
        perplexity = calculate_perplexity(pred, model, tokenizer)

        # Tokenización
        ref_tokens = word_tokenize(ref)
        pred_tokens = word_tokenize(pred)

        # Padding the shorter sequence
        max_length = max(len(ref_tokens), len(pred_tokens))
        ref_tokens += [''] * (max_length - len(ref_tokens))  # Pad with empty strings
        pred_tokens += [''] * (max_length - len(pred_tokens))

        # Exactitud (Accuracy) a nivel de token
        accuracy = accuracy_score(ref_tokens, pred_tokens)

        # Precisión, Recuperación y Medida F1 a nivel de token
        precision = precision_score(ref_tokens, pred_tokens, average='macro', zero_division=0)
        recall = recall_score(ref_tokens, pred_tokens, average='macro', zero_division=0)
        f1 = f1_score(ref_tokens, pred_tokens, average='macro', zero_division=0)

        # BLEU
        bleu = load_metric('sacrebleu')
        bleu_score = bleu.compute(predictions=[pred], references=[[ref]])['score']

        # ROUGE
        rouge = load_metric('rouge')
        rouge_score = rouge.compute(predictions=[pred], references=[ref])

        results.append({
            'reference': ref,
            'prediction': pred,
            'Perplexity': perplexity,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'BLEU': bleu_score,
            'ROUGE': rouge_score,
            'inference_time': inf
        })

    return pd.DataFrame(results)

# Evaluar ambos modelos

In [ ]:
# Evaluar ambos modelos
results_llama_2 = evaluate_model_individual(data, 'llama_2')
results_gpt = evaluate_model_individual(data, 'gpt')

# Guardar resultados en archivos CSV
results_llama_2.to_csv('results_llama_2_' + temperature + '.csv', index=False)
results_gpt.to_csv('results_gpt_' + temperature + '.csv', index=False)

# Mostrar algunos resultados como ejemplo
print("Resultados de LLaMa 2 (primeros 5 registros):")
display(results_llama_2.head())

print("\nResultados de gpt-3.5-turbo (primeros 5 registros):")
display(results_gpt.head())

Resultados de LLaMa 2 (primeros 5 registros):


,reference,prediction,Perplexity,Accuracy,Precision,Recall,F1 Score,BLEU,ROUGE,inference_time
0,En el TRL 1 se realizan actividades de investi...,En el TRL 1 se realizan actividades de invest...,47.015709,0.322581,0.258065,0.301075,0.270968,31.090657,"{'rouge1': ((0.47058823529411764, 0.64, 0.5423...",39.129276
1,En el TRL 7 se realizan pruebas en condiciones...,En el TRL 7 se realizan pruebas en condicione...,23.081341,0.136986,0.183761,0.243590,0.199634,13.608933,"{'rouge1': ((0.22077922077922077, 0.7727272727...",38.132498
2,La finalización del TRL 8 se demuestra cuando ...,La finalización del TRL 8 se demuestra cuando...,64.385033,0.227273,0.153595,0.196078,0.166667,27.906680,"{'rouge1': ((0.4318181818181818, 0.5, 0.463414...",38.120561
3,Los principales resultados del TRL 6 son el pr...,Los resultados del TRL 6 son prototipos de pr...,16.386143,0.014706,0.026316,0.026316,0.026316,6.551117,"{'rouge1': ((0.16901408450704225, 0.5454545454...",38.552591
4,El TRL 5 representa la prueba del prototipo de...,El TRL 5 representa la validación de los comp...,25.697437,0.083333,0.122449,0.142857,0.125000,7.368827,"{'rouge1': ((0.19402985074626866, 0.8125, 0.31...",37.978098



Resultados de gpt-3.5-turbo (primeros 5 registros):


,reference,prediction,Perplexity,Accuracy,Precision,Recall,F1 Score,BLEU,ROUGE,inference_time
0,En el TRL 1 se realizan actividades de investi...,En el TRL 1 se realizan actividades de investi...,76.031242,0.461538,0.402778,0.444444,0.416667,48.443290,"{'rouge1': ((0.7857142857142857, 0.88, 0.83018...",0.696570
1,En el TRL 7 se realizan pruebas en condiciones...,En el TRL 7 se realizan pruebas en condiciones...,100.634628,0.900000,0.868421,0.868421,0.859649,80.315467,"{'rouge1': ((0.9047619047619048, 0.86363636363...",0.584482
2,La finalización del TRL 8 se demuestra cuando ...,La finalización del TRL 8 se demuestra cuando ...,76.344948,0.324324,0.255556,0.255556,0.251852,30.583918,"{'rouge1': ((0.6428571428571429, 0.47368421052...",0.625681
3,Los principales resultados del TRL 6 son el pr...,Los principales resultados del TRL 6 son la va...,104.182526,0.205882,0.180556,0.180556,0.180556,22.168238,"{'rouge1': ((0.3888888888888889, 0.63636363636...",1.015756
4,El TRL 5 representa la prueba del prototipo de...,El TRL 5 representa la validación de los compo...,98.288483,0.421053,0.320000,0.320000,0.320000,26.801652,"{'rouge1': ((0.5, 0.625, 0.5555555555555556), ...",0.650612


#  Cálculo de puntajes promedio

In [ ]:
def calculate_average_scores(results):
    average_scores = {
        'Perplexity': results['Perplexity'].mean(),
        'Accuracy': results['Accuracy'].mean(),
        'Precision': results['Precision'].mean(),
        'Recall': results['Recall'].mean(),
        'F1 Score': results['F1 Score'].mean(),
        'BLEU': results['BLEU'].mean(),
        'ROUGE': {
            'rouge1': results['ROUGE'].apply(lambda x: x['rouge1'].mid.fmeasure).mean(),
            'rouge2': results['ROUGE'].apply(lambda x: x['rouge2'].mid.fmeasure).mean(),
            'rougeL': results['ROUGE'].apply(lambda x: x['rougeL'].mid.fmeasure).mean(),
        },
        'inference_time': results['inference_time'].mean()
    }
    return average_scores

average_scores_llama_2 = calculate_average_scores(results_llama_2)
average_scores_gpt = calculate_average_scores(results_gpt)

# Combinar resultados individuales y promedio en un solo DataFrame
results_combined = pd.DataFrame([average_scores_llama_2, average_scores_gpt], index=['LLaMa 2', 'GPT 3.5 Turbo']) \
                     .reset_index().rename(columns={'index': 'Model'})
average_scores_df = pd.DataFrame({
    'Model': ['LLaMa 2', 'GPT 3.5 Turbo'],
    'Perplexity': [average_scores_llama_2['Perplexity'], average_scores_gpt['Perplexity']],
    'Accuracy': [average_scores_llama_2['Accuracy'], average_scores_gpt['Accuracy']],
    'Precision': [average_scores_llama_2['Precision'], average_scores_gpt['Precision']],
    'Recall': [average_scores_llama_2['Recall'], average_scores_gpt['Recall']],
    'F1 Score': [average_scores_llama_2['F1 Score'], average_scores_gpt['F1 Score']],
    'BLEU': [average_scores_llama_2['BLEU'], average_scores_gpt['BLEU']],
    'ROUGE-1': [average_scores_llama_2['ROUGE']['rouge1'], average_scores_gpt['ROUGE']['rouge1']],
    'ROUGE-2': [average_scores_llama_2['ROUGE']['rouge2'], average_scores_gpt['ROUGE']['rouge2']],
    'ROUGE-L': [average_scores_llama_2['ROUGE']['rougeL'], average_scores_gpt['ROUGE']['rougeL']],
    'inference_time': [average_scores_llama_2['inference_time'], average_scores_gpt['inference_time']]
})

# Guardar resultados combinados en un archivo CSV
results_combined.to_csv('results_combined_' + temperature + '.csv', index=False)

# Mostrar los puntajes promedio en forma de tabla
print("\nPuntajes Promedio de Cada Métrica para Ambos Modelos:")
display(average_scores_df)


Puntajes Promedio de Cada Métrica para Ambos Modelos:


,Model,Perplexity,Accuracy,Precision,Recall,F1 Score,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,inference_time
0,LLaMa 2,50.205136,0.139561,0.131178,0.153977,0.136837,14.290513,0.389980,0.227978,0.343217,38.351553
1,GPT 3.5 Turbo,172.347141,0.344405,0.297353,0.296840,0.294447,35.852842,0.604466,0.452481,0.562019,0.943709
